In [ ]:
基于本地基础镜像构建新镜像

In [ ]:
1. Docker 构建镜像的工作流程：
step1. 本地查找
当执行 docker build 时，Docker 会首先在本地镜像仓库中查找 ubuntu:20.04
step2. 自动下载（如果本地不存在）
如果本地没有该镜像，Docker 会自动从 Docker Hub 拉取官方仓库的 ubuntu:20.04 镜像,如果dockerfile中定义了镜像网站，则会从镜像网站拉取
step3. 构建继续
无论镜像来自本地还是远程下载，后续构建过程都会正常进行
*********************************************************************
即：
（1）无论基于本地还是下载的，都可以写 FROM，没有任何区别！
（2）如果镜像是从网页上下载的，可以 docker load 一下，将镜像加载到本地仓库，

In [ ]:
1.5 本地镜像处理
（1）必须使用 docker load 加载镜像
从网页下载的镜像文件（通常是 .tar 格式），必须通过 docker load 命令加载到本地仓库。
直接移动文件到 Docker 存储目录会导致镜像无法被正确识别。
# 进入下载文件所在目录
cd ~/Downloads
# 加载镜像（假设文件名为 image.tar）
docker load -i image.tar
# 验证加载结果
docker images
（2）不能直接移动文件到仓库目录
** Docker 的镜像存储结构复杂：镜像采用分层存储机制，每个层有独立的哈希标识，手动操作会破坏元数据关联。
** 存储驱动依赖：Docker 使用 overlay2、aufs 等存储驱动管理文件系统，直接修改底层文件可能导致不可预知的错误。
** 权限问题：Docker 目录需要严格的权限控制，手动操作可能引发权限冲突。
（3）Docker 本地仓库目录位置
/var/lib/docker
包含以下关键子目录：image/:镜像元数据；overlay2/:分层存储的实际文件（如果使用overlay2驱动）；containers/:容器数据
（4）重要警告
禁止手动操作 /var/lib/docker：即使你知道目录结构，直接修改或添加文件可能导致：镜像损坏，Docker 服务崩溃，数据丢失
如果确实需要迁移 Docker 数据，应使用以下方法：
    # 停止 Docker 服务
    sudo systemctl stop docker
    # 备份整个 Docker 目录
    sudo rsync -a /var/lib/docker /path/to/backup
    # 恢复时同样通过文件复制+重启服务
（5）镜像管理推荐方式
操作        正确命令                       错误做法
加载镜像    docker load -i file.tar       复制文件到 /var/lib/docker
保存镜像    docker save -o file.tar       从 /var/lib/docker 直接打包
删除镜像    docker rmi <镜像ID>            手动删除仓库目录中的文件
（6）镜像存储原理（分层存储结构）：
    /var/lib/docker/overlay2/
    ├── l/          # 层符号链接
    ├── <layer_id>  # 具体层目录
    │   ├── diff    # 层内容
    │   └── link    # 哈希标识
    └── imagedb     # 镜像数据库
每个镜像由多个只读层（layer）组成，这些层通过联合文件系统（如 overlay2）叠加为容器提供最终视图。
（7）总结
永远通过 docker load 加载外部镜像，使用 docker images 和 docker rmi 管理镜像；
不要手动操作 /var/lib/docker 目录，不要尝试直接移动文件到 Docker 存储路径。

In [ ]:
2. 准备工作
# 创建项目目录
mkdir my_docker_project && cd my_docker_project
# 准备代码文件
将需要打包的代码（如 app 目录）和依赖文件（如 requirements.txt ）放入项目目录，创建dockerfile：
my_docker_project/
├── app/
│   ├── main.py
│   └── ...
├── requirements.txt
└── Dockerfile
# 创建.dockerignore 文件（可选）避免将无关文件（如虚拟环境、日志）复制到镜像中：
.git
__pycache__
*.log
venv/

In [ ]:
3. 编写 Dockerfile
# 使用官方 Ubuntu 20.04 镜像作为基础
FROM ubuntu:20.04
# 设置环境变量（避免交互式询问时卡住）
ENV DEBIAN_FONTEND=noninteractive
# 更新系统并安装基础工具
RUN apt-get update && \
    apt-get install -y \
    python3 \
    python3-pip \
    && rm -rf /var/lib/apt/lists/*
# 设置工作目录
WORKDIR /app
# 复制本地代码和依赖文件到镜像中
COPY requirements.txt .
COPY ./app ./app
# 安装 Python 依赖库
RUN pip3 install --no-cache-dir -r requirements.txt
# 暴露端口（根据实际应用调整）
EXPOSE 8000
# 定义容器启动命令（示例为启动 Python 应用）
CMD ["python3", "./app/main.py"]

In [ ]:
4. 构建镜像
# 执行构建命令
docker build -t my-custom-image:v1 . # -t 指定镜像名称和标签，. 表示使用当前目录的 Dockerfile
# 查看构建结果
docker images | grep my-custom-image
(1) | 管道符
作用：将前一个命令的输出结果传递给后一个命令作为输入
docker images | grep myapp
# | 将docker images输出的列表，传递给 grep myapp 
(2) grep 
作用：在文本中搜索匹配的字符串（grep myapp 表示搜索包含 myapp 的行）
docker images | grep myapp
# grep myapp 从 docker images 的输出中筛选出包含 myapp 的行（通常是镜像名或标签）。

In [ ]:
5. 验证镜像
# 运行容器
docker run -it --rm my-custom-image:v1 # --rm：容器退出后自动删除（测试时常用），如需后台运行：添加-d
# 进入容器检查
docker run -it --entrypoint /bin/bash my-custom-image:v1 
# 验证文件是否复制成功：ls/app，检查依赖是否安装：pip3 list

In [ ]:
6. 高级定制技巧
（1）优化镜像层
* 将高频变化的操作（如代码复制）放在 Dockerfile 末尾；
* 合并 RUN 指令减少层数（如果报错时，可以拆分RUN指令用来调试，调试完成后还是建议合并RUN指令）
RUN apt-get update && \
    apt-get install -y git curl && \
    apt-get clean
（2）多阶段构建（适用于编译型语言，比如需要make的）
# 第一阶段：构建环境
FROM ubuntu:20.04 as builder
RUN apt-get update && apt-get install -y build-essential
COPY src /src
RUN make /src
- as builder 在 Dockerfile 中用于定义「多阶段构建」的构建阶段别名，具体作用如下：
- 阶段标识：给当前构建阶段命名（此处为 builder）
- 隔离环境：创建独立的构建环境，避免污染最终镜像
- 资源优化：允许后续阶段仅复制必要的构建产物
关键优势：减小镜像体积-最终镜像不包含编译工具链等临时依赖；提升安全性-生产环境镜像不包含开发调试工具；优化构建缓存-修改代码时无需重新安装基础依赖
操作原理：第一阶段 (builder) 完成编译/构建操作；第二阶段使用基础镜像重新开始构建；通过 COPY --from=builder 仅复制需要的文件
扩展用法1：as builder 可以有多个，用来定义多个构建阶段，FROM node:14 as frontend-builder，FROM golang:1.18 as backend-builder
扩展用法2：动态选择阶段：通过 --target 参数指定构建阶段
# 第二阶段：运行环境
FROM ubuntu:20.04
COPY --from=builder /src/output /app
CMD ["/app/myapp"]
（3）配置镜像元数据
LABEL maintainer="yourname@example.com"
LABEL version="1.0"

In [ ]:
7. 更新镜像流程
（1）修改代码或requirements.txt
（2）重新构建镜像：docker build -t my-custom-image:v2 .
8. 关键注意事项
（1）镜像最小化原则：
    删除不必要的缓存文件：apt-get install && rm -rf /var/lib/apt/lists/*
    使用 .dockerignore 减少上下文大小
（2）时区设置
    RUN ln -sf /usr/share/zoneinfo/Asia/Shanghai /etc/localtime
（3）依赖管理
    固定版本号：pip install package==1.0.0
    定期更新基础镜像安全补丁